<a href="https://colab.research.google.com/github/Gyawalisahaj/Deeplearn/blob/main/LSTMWordPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, dataloader
import nltk
from nltk.tokenize import word_tokenize

In [3]:
document = """About Data Science Mentorship Program
What is the course fee for  Data Science Mentorship Program (DSMP 2.0)
The total course fee for the DSMP course is Rs 10,600. This includes the fee for both DSMP 1.0 and DSMP 2.0
What is the total duration of the course?
The total duration of the course is about 6-8 months.
Are Deep Learning and NLP a part of the DSMP 2.0  course?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes, all our sessions are recorded, so even if you miss a session you can login into our portal and watch the recording as per your convenience.
Where can I find the class schedule?
You will find the class schedule in your course dashboard once you enroll for the course.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish.
How will I be informed about the upcoming class?
You will get mail from our side before every session once you enroll in the course.
Can I do this course if I am from a non-tech background?
Yes, absolutely. However, you’ll need to be consistent.
I am late, can I join the program in the middle?
Absolutely, you can join the program any time.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you’re enrolled in the course, you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes, there are 5 end to end case studies and multiple smaller projects.
Do we offer any student discounts or promo codes?
We do offer discounts on our courses a few times a year, and we announce them through our social media channels.
On how many devices can I watch the videos simultaneously?
You can watch the videos on one device at a time.
Do you provide notes for the lectures?
We do not provide notes for our lectures. We encourage students to make their own notes as it aids in study retention.
What is the difference between being a paid member on your YouTube channel and your website?
While the content on our YouTube channel and website is similar, enrolling through our website offers additional benefits such as doubt clearance support and access to an exclusive community of learners on Discord and more.
Do I get access to your YouTube channel as a paid member after enrolling through your website?
No, the YouTube channel membership is separate from what we offer on our website.
Is there any upcoming batch for the Data Science Mentorship Program?
Currently, we do not have any plans to launch a new batch for DSMP. However, all previous lectures are available as recordings, allowing you to learn at your own pace.
Do you provide doubt clearance for the projects uploaded on CampusX YouTube channel?
We do not provide doubt clearance for any of our videos or projects uploaded on YouTube. However, you can ask your doubts in our Discord channel or in the comments section.
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
What if I don’t like the course after making the payment? What is the refund policy?
You get a 7 days refund period from the day you have made the payment. To apply for refund, send us an email at support@campusx.in
I am living outside India and I am not able to make the payment on the website. What should I do?
You can contact us by sending a mail at support@campusx.in
I am from Pakistan and I am unable to make the payment?
We currently do not have the option to accept payment from Pakistan directly. If you’d still like to enroll for the course, you may ask a friend or family member living outside Pakistan to make the fee payment on your behalf.
Do you accept PayPal as a payment method?
Yes, we do. Please contact us at support@campusx.in for more details on using PayPal for payment.
I cannot pay the DSMP 2.0 fee as a one-time payment, is there any way to make a payment in installments?
At the moment, we do not have a monthly payment plan for the DSMP 2.0 course. However, if you have financial constraints, you can enroll in DSMP 1.0 by paying 799/- a month. Once you complete the course, you may apply for a discount coupon from the website.
Post registration queries
Till when can I view the paid videos on the website?
You can watch the videos till your subscription is valid, that is 3 years from the date of purchase for DSMP 2.0 and 2 years for DSMP 1.0
Why is lifetime validity not provided?
Because of the low course fee.
Do you assist with personal projects?
We do not provide support for personal projects due to time constraints and current commitments.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearing session.
If I join the program late, can I still ask past week doubts?
Yes, just select “past week's doubt” in the doubt clearance google form.
What is the validity of Doubt Support?
You will get doubt support for 1 year from the data of registration.
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 10,600
You have to attempt all the course assessments.
I have read that Job Preparation is a part of this program. What comes under Job Preparation?
This is to clarify that Job Preparation does not mean Placement guarantee. We don't guarantee our learners any jobs or for that matter even interview calls. If you are planning to join this course just for placements, we’re afraid that this course might not be the best for you.
Here is what comes under Job Preparation
Portfolio Building sessions.
Interview Questions.
Sessions with industry mentors.
Discussion on Job hunting strategies."""


In [4]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
tokens = word_tokenize(document.lower())

In [6]:
vocab = {'<unk>':0}
for token in Counter(tokens).keys():
  if token  not in vocab:
    vocab[token] = len(vocab)

vocab

{'<unk>': 0,
 'about': 1,
 'data': 2,
 'science': 3,
 'mentorship': 4,
 'program': 5,
 'what': 6,
 'is': 7,
 'the': 8,
 'course': 9,
 'fee': 10,
 'for': 11,
 '(': 12,
 'dsmp': 13,
 '2.0': 14,
 ')': 15,
 'total': 16,
 'rs': 17,
 '10,600.': 18,
 'this': 19,
 'includes': 20,
 'both': 21,
 '1.0': 22,
 'and': 23,
 'duration': 24,
 'of': 25,
 '?': 26,
 '6-8': 27,
 'months': 28,
 '.': 29,
 'are': 30,
 'deep': 31,
 'learning': 32,
 'nlp': 33,
 'a': 34,
 'part': 35,
 'no': 36,
 ',': 37,
 'not': 38,
 '’': 39,
 's': 40,
 'curriculum': 41,
 'if': 42,
 'i': 43,
 'miss': 44,
 'live': 45,
 'session': 46,
 'will': 47,
 'get': 48,
 'recording': 49,
 'yes': 50,
 'all': 51,
 'our': 52,
 'sessions': 53,
 'recorded': 54,
 'so': 55,
 'even': 56,
 'you': 57,
 'can': 58,
 'login': 59,
 'into': 60,
 'portal': 61,
 'watch': 62,
 'as': 63,
 'per': 64,
 'your': 65,
 'convenience': 66,
 'where': 67,
 'find': 68,
 'class': 69,
 'schedule': 70,
 'in': 71,
 'dashboard': 72,
 'once': 73,
 'enroll': 74,
 'time': 75,
 '

In [7]:
input_sentences = document.split('\n')
input_sentences

['About Data Science Mentorship Program',
 'What is the course fee for  Data Science Mentorship Program (DSMP 2.0)',
 'The total course fee for the DSMP course is Rs 10,600. This includes the fee for both DSMP 1.0 and DSMP 2.0',
 'What is the total duration of the course?',
 'The total duration of the course is about 6-8 months. ',
 'Are Deep Learning and NLP a part of the DSMP 2.0  course?',
 'No, NLP and Deep Learning both are not a part of this program’s curriculum.',
 'What if I miss a live session? Will I get a recording of the session?',
 'Yes, all our sessions are recorded, so even if you miss a session you can login into our portal and watch the recording as per your convenience.',
 'Where can I find the class schedule?',
 'You will find the class schedule in your course dashboard once you enroll for the course.',
 'What is the time duration of all the live sessions?',
 'Roughly, all the sessions last 2 hours.',
 'What is the language spoken by the instructor during the session

In [8]:
def text_to_indices(sentence, vocab):
  numerical_sentence = []
  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence

In [9]:
input_numerical_sentences = []
for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))

In [10]:
len(input_numerical_sentences)

79

In [11]:
training_sequence = []
for sentence in input_numerical_sentences:
  for i in range(1, len(sentence)):
    training_sequence.append(sentence[:i+1])

In [12]:
len(training_sequence)

1178

In [13]:
training_sequence[:5]

[[1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [6, 7]]

In [15]:
len_list = []
for sequence in training_sequence:
  len_list.append(len(sequence))

  max(len_list)

In [16]:
training_sequence[0]

[1, 2]

In [19]:
padded_traning_sequence = []
max_len = max(len_list)
for sequence in training_sequence:
  padding_length = max_len - len(sequence)
  padded_sequence = [0] * padding_length + sequence
  padded_traning_sequence.append(padded_sequence)
print(padded_traning_sequence)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 7], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [20]:
padded_traning_sequence = torch.tensor(padded_traning_sequence, dtype=torch.long)

In [22]:
padded_traning_sequence

tensor([[  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        [  0,   0,   0,  ...,   2,   3,   4],
        ...,
        [  0,   0,   0,  ..., 143, 327, 350],
        [  0,   0,   0,  ..., 327, 350, 351],
        [  0,   0,   0,  ..., 350, 351,  29]])

In [23]:
X = padded_traning_sequence[:,:-1]
y = padded_traning_sequence[:,-1]


In [24]:
X

tensor([[  0,   0,   0,  ...,   0,   0,   1],
        [  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        ...,
        [  0,   0,   0,  ..., 349, 143, 327],
        [  0,   0,   0,  ..., 143, 327, 350],
        [  0,   0,   0,  ..., 327, 350, 351]])

In [25]:
y

tensor([  2,   3,   4,  ..., 350, 351,  29])

In [26]:
from os import XATTR_SIZE_MAX
class costumDataset(Dataset):
  def __init__(self,X ,y):
    self.X = X
    self.y = y

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [27]:
dataset = costumDataset(X,y)


In [28]:
len(dataset)

1178

In [30]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [31]:
for input , output in dataloader:
  print(input)
  print(output)
  break

tensor([[  0,   0,   0,  ..., 172, 173,  34],
        [  0,   0,   0,  ...,  57,  39, 114],
        [  0,   0,   0,  ...,   0,   0,  57],
        ...,
        [  0,   0,   0,  ...,  32,  23,  33],
        [  0,   0,   0,  ...,  57,  58,  59],
        [  0,   0,   0,  ...,  29,   6, 246]])
tensor([174, 115, 117,   2,   7,   8,  37, 247,  26,  26,  48,  25,  26, 220,
         38,   9,  62,  23,  40, 347,  37, 108,  14,  52, 164, 220, 242,  70,
         37,  34,  60,  43])


In [32]:
dataset[0]

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1]),
 tensor(2))